# Segunda Avaliação Parcial

Neste módulo trabalhamos a visualização de dados, assim como as técnicas para acessar estes dados da internet. As técnicas de web scraping que estudamos, usando __Requests__, __Beautiful Soup__ e __Selenium__, permitem implementar aplicações que processam informação diretamente da rede.  

Desde a Instrução prática 7, por exemplo, estamos trabalhando com dados que baixamos do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos).  

Aprendemos também como implementar uma aplicação com GUI utilizando o __TK Inter__. 

Neste contexto vamos implementar uma aplicação que, a traves de uma GUI, permita gerar gráficos de precipitações e temperaturas medias mensais de um determinado ano, para uma estação meteorológica. A atividade deve ser organizada da seguinte forma.

## Exercício 1

__Primeiro protótipo__: Desenvolva uma interface gráfica que pegue, em tempo real, as informações sobre os dados de quais anos estão disponíveis no site do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). Os anos disponíveis devem ser apresentados utilizando um widget apropriado que permita selecionar um, e apenas um ano. Se não tiver conexão a Internet ou se o site não estiver disponível o aplicativo deve mostrar uma mensagem de erro na interface.


In [ ]:
%run -m metereologico_V1

<h1>Explicação</h1>
No primeiro protótipo foram criadas três classes. 
A primeira, intitulada de "BuscaDados", dedicada à buscar os dados requeridos a partir da url informada, conta com a utilização de bibliotecas como a requests, beautifulSoup, HTTPError e re. Esta classe possui um método que busca os dados do INMET, faz tratamento de exceções e retorna uma lista de tuplas contendo os anos disponíveis (formatados com o re.search()) no site e seus respectivos links, além de uma variável contendo um erro, caso haja exceção.

In [ ]:
# Define a classe BuscaDados
class BuscaDados:

    # Construtor da classe com parâmetros anos e erro
    def __init__(self, anos, erro):
        self.__anos = anos 
        self.__erro = erro  


    def buscaDados(self):
        # Define o cabeçalho para simular um navegador ao enviar a requisição
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        url = "https://portal.inmet.gov.br/dadoshistoricos"  # URL do portal de dados históricos

        self.__anos = []
        self.__erro = None

        try:
            # Envia uma requisição HTTP GET para a URL
            resposta = requests.get(url, headers=headers)
            resposta.raise_for_status()  # Lança uma exceção se a resposta contiver um erro HTTP
        except HTTPError as http_err:
            self.__erro = http_err  
        except Exception as err:
            self.__erro = err  
        else:
            respostaHtml = BeautifulSoup(resposta.text, 'html.parser') # retorna um html
            anosTag = respostaHtml.find_all('article')  # Encontra todas as tags 'article' no HTML
            for ano in anosTag:
                # Usa expressão regular para extrair o ano de uma string
                anoFormat = re.search(r'\d{4}', ano.find('a').string)
                # Adiciona uma tupla contendo o ano e o link correspondente à lista de anos
                self.__anos.append((anoFormat.group(), ano.find('a')['href']))

        return self.__anos, self.__erro


Na segunda classe, intitulada "Interface", utiliza-se a biblioteca tkinter para a geração da janela que irá interagir com o usuário.

In [ ]:
def interface(self):
        # Cria uma instância do objeto Tk, que representa a janela principal da interface
        app = tk.Tk()

        # Configurações iniciais da janela
        app.title("Dados Meteorológicos")
        app.geometry('500x300')  
        app.configure(background='#c38680')  

        self.anos, self.erro = self.busca.buscaDados()

        # Verifica se ocorreu algum erro durante a busca
        if self.erro:
            # Cria um rótulo para exibir a mensagem de erro na interface
            mensagemErro = tk.Label(app, text=f"Erro ao obter dados: {self.erro}", background='#c38680', wraplength=450)
            mensagemErro.place(x=10, y=70, width=450, height=30)
        else:
            # Se não houver erro, cria um menu suspenso com os anos disponíveis
            opcao = tk.StringVar(app)
            opcao.set("ANO")
            texto1 = tk.OptionMenu(app, opcao, *[ano[0] for ano in self.anos])
            texto1.place(x=10, y=10, width=100, height=50)  # Define a posição e o tamanho do menu suspenso

        # Inicia o loop principal da interface gráfica
        app.mainloop()

## Exercício 2

__Segundo protótipo__: Com base no ano escolhido sua aplicação deve baixar o arquivo compactado, com os dados de todas as estações, descompactar ele numa pasta temporária e mostrar em um outro widget a lista de estações disponíveis para selecionar uma e apenas uma delas. O widget do protótipo anterior deve ficar disponível para que o usuário possa trocar o ano se desejar. Nesse caso um novo arquivo será baixado e descompactado e as estações disponíveis exibidas para seleção.

In [ ]:
%run -m metereologico_V2

No segundo protótipo, o botão que contém as opções dos anos disponíveis emite um comando para a função escolhaCidades que por sua vez chama uma nova classe "DownloadDados" para receber as ciades disponíveis e mostrá-las em m novo botão.

In [ ]:
#....
texto1 = tk.OptionMenu(app, opcao, *[ano[0] for ano in self.anos],
                                    command=lambda anoSelecionado: self.escolhaCidades(anoSelecionado, app))
texto1.place(x=10, y=10, width=100, height=50) 
app.mainloop()

    # Método para escolher cidades após a seleção de um ano
    def escolhaCidades(self, anoSelecionado, app):
        # Chama o método downloadDados da classe DownloadDados para obter as cidades disponíveis
        self.cidades = self.down.downloadDados(anoSelecionado, self.anos)
        opcao2 = tk.StringVar(app)
        opcao2.set("CIDADE")
        # Cria um novo menu suspenso com as cidades disponíveis
        texto2 = tk.OptionMenu(app, opcao2, *self.cidades)
        texto2.place(x=10, y=70, width=150, height=50)  

Enquanto isso, na classe DownloadDados, possui um método chamado downloadDados que baixa e extrai dados para o ano selecionado. Neste, utiliza-se a biblioteca wget para baixar o arquivo zip do link correspondente ao ano e a zipfile para extrair o conteúdo do arquivo zip na pasta 'csvs'. Em seguida, itera sobre os títulos dos arquivos extraídos para obter os nomes das cidades formatados e retorna a lista de nomes de cidades..

In [ ]:
class DownloadDados:

    # Método para baixar dados para o ano selecionado
    def downloadDados(self, anoSelecionado, anos):
        link = None

        # Procura o link correspondente ao ano selecionado
        for ano in anos:
            if ano[0] == anoSelecionado:
                link = ano[1]
                # Faz o download do arquivo zip para a pasta 'csvs' com o nome do ano selecionado
                wget.download(link, f'csvs/{anoSelecionado}.zip')
        
        # Define o caminho do arquivo zip baixado
        caminho = f'csvs/{anoSelecionado}.zip'
        
        # Extrai o conteúdo do arquivo zip para a pasta 'csvs'
        with zipfile.ZipFile(caminho, 'r') as zip_ref:
            zip_ref.extractall(f'csvs')
            titulosArquivos = zip_ref.namelist()
            
            nomesCidades = []
            for titulo in titulosArquivos:
                partes = titulo.split('_')
                if len(partes) >= 5:
                    cidade = partes[4]
                    nomesCidades.append(cidade)
        return nomesCidades

# Exercício 3

__Terceiro protótipo__: Com base na estação selecionada gere e exiba dentro da aplicação os gráficos de precipitações e temperaturas medias mensais para a estação selecionada. A interface deve permitir que o usuário troque a estação selecionada a qualquer momento, assim como foi feito co o ano, gerando novos gráficos.   

In [ ]:
%run -m metereologico_V3/metereologico/

No terceiro prótótipo a principal mudança é a adição da classe Graficos. A classe recebe os titulos obtidos no módulo _downloadDados e utiliza para criar o caminho que será usado para gerar o dataFrame. Em seguida, ocorre um tratamento dos dados para enfim criar os gráficos utilizando subplots
O método tratamentoDados realiza algumas manipulações no DataFrame. Além disso, pequenos ajustes foram feitos na classe Interface, para receber os gráficos plotados e ajustá-los à janela da aplicação.

In [ ]:
#....    
    # Método para criar um DataFrame a partir dos dados baixados
    def dataFrame(self, cidade, anoSelecionado, anos):
        self.dow = _downloadDados.DownloadDados()
        tituloArquivos = self.dow.downloadDados(anoSelecionado, anos)[1]
        for titulo in tituloArquivos:
            if cidade in titulo:
                caminhoCSV = f'csvs/{titulo}'
        
        # Lê o arquivo CSV e cria um dataframe
        self.__df = pd.read_csv(caminhoCSV, encoding="iso-8859-1", decimal=',', sep=';', skiprows=8)

    # Método para realizar tratamento nos dados do DataFrame
    def tratamentoDados(self):
        self.__df.rename({"DATA (YYYY-MM-DD)":"Data","HORA (UTC)":"Hora","Hora UTC":"Hora"}, axis=1, inplace=True)
        self.__df.index = pd.DatetimeIndex(self.__df["Data"]+" "+self.__df["Hora"], name="DATA")
        self.__df.loc[self.__df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].abs() > 1000, "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = None
        self.__df.loc[self.__df["TEMPERATURA DO PONTO DE ORVALHO (°C)"].abs() > 1000, "TEMPERATURA DO PONTO DE ORVALHO (°C)"] = None
    
    # Método para plotar gráficos de temperatura média mensal e precipitação média mensal
    def plotGrafTemp(self):
        # Calcula a média mensal da temperatura do ponto de orvalho
        df = self.__df.groupby(self.__df.index.month)["TEMPERATURA DO PONTO DE ORVALHO (°C)"].mean()
        
        # Configuração do primeiro subplot (Temperatura Média Mensal)
        plt.clf()
        plt.subplot(2,1,1)
        plt.plot(df.index, df, marker='s')
        plt.title('Temperatura Média Mensal')
        plt.xlabel('Mês')
        plt.ylabel('Temperatura (°C)')
        plt.grid(True)

        # Calcula a média mensal da precipitação total horária
        dfT = self.__df.groupby(self.__df.index.month)["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].mean()
        
        # Configuração do segundo subplot (Precipitação Média Mensal)
        plt.subplot(2,1,2)
        plt.plot(dfT.index, dfT, marker='s')
        plt.title('Precipitação Média Mensal')
        plt.xlabel('Mês')
        plt.ylabel('Temperatura (°C)')
        plt.grid(True)
        plt.subplots_adjust(wspace=0, hspace=1)

# Função para criar uma instância de Graficos, baixar dados, realizar tratamento e plotar gráficos
def plotGrafico(cidade, anoSelecionado, anos):
    df = Graficos()
    df.dataFrame(cidade, anoSelecionado, anos)
    df.tratamentoDados()
    df.plotGrafTemp()


## Respostas

Implemente sua aplicação como um pacote __Python__ e disponibilize neste __Notebook__ um explicação dos principais pontos da implementação. Utilize o comando mágico ``%run`` para executar o aplicativo desenvolvido.  Envie o __Notebook__ via __Moodle__ assim como o repositório com a implementação do aplicativo até o final do prazo. 